In [ ]:
import os
import json
import pandas as pd
import networkx
import numpy as np
import obonet  # conda install -c biobuilds obonet

In [ ]:
go_obo_file = "../data/disprot/go-basic.obo"

# Mongo collections
disprot_old_file = "../data/disprot/disprot8.entries_2023_12.json"
#disprot_new_file = "../data/disprot/disprot8.entries_2024_06_c.json"
disprot_new_file = "../data/disprot/disprot8.entries_2024_12_c.json"  # 29 October 2024 (10:15 CET)

# Download from SIFTS
sifts_file = "../data/sifts/uniprot_segments_observed.tsv.gz"
accession_filter_file = None #"../data/accession_filter.txt"  # Limit the reference to these accessions

# Output
references_dir = "../data/output/references"
dataset_raw_file = "../data/output/dataset_raw.tsv"
dataset_ec_file = "../data/output/dataset_ec.tsv"
challenge_terms_file = "../data/output/challenge_terms.tsv"
dataset_file = "../data/output/dataset.tsv"
fasta_new_file = "../data/output/homology/disprot_new.fasta"
fasta_old_file = "../data/output/homology/disprot_old.fasta"

In [ ]:
def expand_region(df_:pd.DataFrame, start_col:str='start', end_col:str='end', res_col:str='reg_position') -> pd.DataFrame:
    df_[res_col] = list(range(int(df_[start_col]), int(df_[end_col]) + 1, 1))
    return df_

def expand_sequence(df_:pd.DataFrame, seq_column:str='sequence', res_col:str='seq_aa') -> pd.DataFrame:
    df_[res_col] = [(i+1, aa) for i, aa in enumerate(df_[seq_column])]
    return df_

## Associate DisProt annotation terms to CAID challenges

In [ ]:
# IDPO terms
data_idpo = [
            ('IDPO:00076', 'disorder', 'disorder'), ('IDPO:00077', 'disorder', 'molten-globule'), ('IDPO:00078', 'disorder', 'pre-molten-globule'), 
    
            ('IDPO:00502', 'linker', 'flexible linker/spacer'),
    
            ('IDPO:00049', 'transition'), 
            ('IDPO:00050', 'transition'), ('IDPO:00051', 'transition'), ('IDPO:00052', 'transition'), ('IDPO:00053', 'transition'), ('IDPO:00060', 'transition'), ('IDPO:00055', 'transition'), 
            ('IDPO:00056', 'transition'), ('IDPO:00061', 'transition'), ('IDPO:00054', 'transition'), ('IDPO:00057', 'transition'), ('IDPO:00058', 'transition'), ('IDPO:00059', 'transition')]

# ('IDPO:00501', 'linker', 'entropic chain'),  ('IDPO:00503', 'linker', 'flexible C-terminal tail'), ('IDPO:00504', 'linker', 'flexible N-terminal tail')

# GO ancestor terms corresponding to CAID2 challenges
ancestors = {'GO:0005488': 'binding', 'GO:0003676': 'binding nucleic acid', 'GO:0005515': 'binding protein'}

In [ ]:
# The OBO must have "ontology: GO" header (first line)
graph = obonet.read_obo(go_obo_file)

# Remove all edges which are not "is_a"
to_remove = []
for e in graph.edges:
    if e[2] != 'is_a':
        to_remove.append((e[0], e[1]))
for ele in to_remove:
    graph.remove_edge(*ele)
    
# Create children table
data_go = []    
for node in graph.nodes(data=True):
    challenge = ancestors.get(node[0])
    if challenge is not None:
        data_go.append([node[0], challenge])
        for children in networkx.ancestors(graph, node[0]): 
            data_go.append([children, challenge])

In [ ]:
df_challenge = pd.DataFrame(data=data_idpo + data_go, columns=['term_id', 'challenge', 'term_name']).drop_duplicates()
df_challenge.to_csv(challenge_terms_file, sep="\t", index=False)
df_challenge

## Process DisProt annotations

In [ ]:
# Get DisProt annotations (mjson format)
# disprot_old = {}
# with open(disprot_old_file, "r") as f:
#     for line in f:
#         obj = json.loads(line)
#         disprot_old[obj["disprot_id"]] = obj
#         
# disprot_new = {}
# with open(disprot_new_file, "r") as f:
#     for line in f:
#         obj = json.loads(line)
#         disprot_new[obj["disprot_id"]] = obj

In [ ]:
# Get DisProt annotations (json format)
disprot_old = {}
with open(disprot_old_file, "r") as f:
     for obj in json.load(f):
         disprot_old[obj["disprot_id"]] = obj
         
# disprot_new = {}
# with open(disprot_new_file, "r") as f:
#      for obj in json.load(f):
#          disprot_new[obj["disprot_id"]] = obj
         
# From a mongoexport of the collection
disprot_new = {}
with open(disprot_new_file, "r") as f:
    for line in f:
        obj = json.loads(line)
        disprot_new[obj["disprot_id"]] = obj

In [ ]:
# Get new annotations (delta = new - old)
dataset = []  # New valid annotations
for disprot_id in disprot_new:
    if disprot_id not in disprot_old and "obsolete" not in disprot_new[disprot_id]:
        if "X" not in disprot_new[disprot_id]["sequence"]:
            # Filter out obsolete regions
            disprot_new[disprot_id]["regions"] = [region for region in disprot_new[disprot_id]["regions"] if "obsolete" not in region]
            if disprot_new[disprot_id]["regions"]:
                dataset.append(disprot_new[disprot_id])
            else:
                print("{} excluded, only obsolete regions".format(disprot_id))
        else:
            print("{} excluded, contains X".format(disprot_id))

In [ ]:
# WARNING
# Filter accession given an input list
if accession_filter_file is not None:
    filter_list = set()
    with open(accession_filter_file) as f:
        for line in f:
            filter_list.add(line.strip())
    dataset = list(filter(lambda x: x['disprot_id'] in filter_list, dataset))
    print(len(dataset))

In [ ]:
# Write fasta for homology calculation
with open(fasta_new_file, "w") as fout:
    for obj in dataset:
        fout.write(">{}|{}\n{}\n".format(obj['disprot_id'], obj['acc'], obj['sequence']))

with open(fasta_old_file, "w") as fout:
    for disprot_id, obj in disprot_old.items():
        if "obsolete" not in obj:
            fout.write(">{}|{}\n{}\n".format(obj['disprot_id'], obj['acc'], obj['sequence']))


In [ ]:
# Convert json to dataframe
entry_columns = ['disprot_id', 'acc', 'ncbi_taxon_id', 'organism', 'sequence']
df = pd.json_normalize(data=dataset, record_path=['regions'], meta=entry_columns, meta_prefix='', record_prefix='')
df = pd.merge(left=df, right=df_challenge, how="inner", on="term_id")
df.to_csv(dataset_raw_file, sep="\t", index=False)
df.columns

In [ ]:
region_columns = ["start", "end", "term_id", "ec_id", "challenge"]
df = df.loc[:, entry_columns + region_columns]
df

In [ ]:
# Get dataset sequences (1 residue per row)
df_sequence = df.apply(expand_sequence, axis=1).copy(deep=True).drop(columns=["ncbi_taxon_id", "organism", "start", "end", "sequence", "term_id", "ec_id", 'challenge'])
df_sequence = df_sequence.explode("seq_aa")
df_sequence[['pos', 'aa']] = pd.DataFrame(df_sequence['seq_aa'].tolist(), index=df_sequence.index)
df_sequence = df_sequence.drop(columns='seq_aa').drop_duplicates()
df_sequence

## Map PDB observed positions using SIFTS

In [ ]:
df_sifts = pd.read_csv(sifts_file, sep="\t", header=1)
# Filter for dataset entries
df_sifts = df_sifts.loc[df_sifts['SP_PRIMARY'].isin(df_sequence['acc'])]
# Explode observed regions 
df_sifts = df_sifts.apply(expand_region, start_col="SP_BEG", end_col="SP_END", axis=1)
df_sifts = df_sifts.explode("reg_position")
df_sifts = df_sifts.loc[:, ['SP_PRIMARY', 'reg_position']].drop_duplicates().reset_index(drop=True).rename(columns={"SP_PRIMARY": "acc"})
df_sifts

In [ ]:
df_sequence = pd.merge(df_sequence, df_sifts, left_on=["acc", "pos"], right_on=["acc", "reg_position"], how="left")
df_sequence.rename(columns={"reg_position": "pdb"}, inplace=True)
df_sequence.loc[df_sequence['pdb'].notnull(), 'pdb'] = 1.0
df_sequence.loc[df_sequence['pdb'].notnull()]

## Define regions

Transform the per-protein dataframe into a per-residue dataframe 


In [ ]:
df_regions = df.apply(expand_region, axis=1).loc[:, ["disprot_id", "reg_position", "ec_id", "challenge"]].copy(deep=True)
# df_regions = pd.merge(left=df_regions, right=df_challenge, how="inner", left_on="term_id", right_on="term_id").drop(columns=["term_id"])
df_regions

In [ ]:
# ECO:0006220, X-ray crystallography-based structural model with missing residue coordinates used in manual assertion  
df_ = df_regions.loc[(df_regions['challenge'] == 'disorder') & (df_regions['ec_id'] != 'ECO:0006220')]
df_.loc[:, 'challenge'] = 'disorder_nox'
df_regions = pd.concat([df_regions, df_])
df_regions

In [ ]:
df_regions_all = df_regions.drop(columns=['ec_id']).explode("reg_position").drop_duplicates()
df_regions_all['has_region'] = 1
df_regions_all

In [ ]:
# Create the pivot table. Transpose challenge values into columns 
df_regions_all = pd.pivot_table(
    df_regions_all,
    columns="challenge",
    index=['disprot_id', 'reg_position'],
    values='has_region')
df_regions_all = df_regions_all.reset_index()
df_regions_all

In [ ]:
# Add sequence positions not mapping to any DisProt region
df_regions_all = pd.merge(left=df_regions_all, right=df_sequence, how="right", left_on=["disprot_id", "reg_position"], right_on=["disprot_id", "pos"])
df_regions_all.drop(columns="reg_position", inplace=True)
df_regions_all

## Write files

Challenge definitions

- The first list are the columns to be considered as positive (any)
- The second list (mask) are the columns to be considered as negative (any)
- If the second list is not provided all non-positives are considered negatives
- In case of conflicts, the positives always overwrite the negatives
- If mask is provided proteins without at least one residue that could be masked (even when overwritten by a positive) are excluded (e.g. only proteins with PDB observed residues are considered) 


In [ ]:
# Reorder the columns
head_cols = ['disprot_id', 'acc', 'pos', 'aa']
disprot_cols = list(df_challenge['challenge'].unique())
other_cols = sorted(list((set(df_regions_all.columns.tolist()) - set(head_cols)) - set(disprot_cols)))
cols = head_cols + disprot_cols + other_cols
df_regions_all = df_regions_all[cols]
df_regions_all

In [ ]:
# Write the dataframe
df_regions_all.to_csv(dataset_file, sep="\t", index=False)

### Write references (Fasta format)

* The next element overwrites the previous in the "class" list
* The "fill" field is used to fill unassigned positions
* Only proteins with at least a "1" are written to file

In [ ]:
challenges = {'linker': {'class': [('linker', '1')], 'fill': '0'}, 
              'linker_disorder': {'class': [('disorder', '0'), ('linker', '1')], 'fill': '-'},
              'disorder': {'class': [('disorder', '1')], 'fill': '0'}, 
              'disorder_nox': {'class': [('disorder_nox', '1')], 'fill': '0'}, 
              'disorder_pdb': {'class': [('pdb', '0'), ('disorder', '1')], 'fill': '-'},
              'disorder_pdb_fill': {'class': [('pdb', '0'), ('disorder', '1')], 'fill': '1'},
              'binding': {'class': [('binding', '1')], 'fill': '0'},
              'binding_nucleic_acid': {'class': [('binding nucleic acid', '1')], 'fill': '0'},
              'binding_disorder': {'class': [('disorder', '0'), ('binding', '1')], 'fill': '-'},
             }

for file_name, challenge in challenges.items():
    with open("{}/{}.fasta".format(references_dir, file_name), "w") as fout:
        for disprot_id, df_g in df_regions_all.groupby('disprot_id'):
            df_g['output'] = np.nan
            # Assign class
            for column, value in challenge['class']:
                df_g.loc[df_g[column].notnull(), 'output'] = value
            # Fill
            if df_g['output'].notnull().any() and challenge.get('fill'):
                df_g.loc[df_g['output'].isnull(), 'output'] = challenge['fill'] 
            # Write proteins with at least one positive assignment
            if (df_g['output'] == '1').any():
                fout.write(">{}\n{}\n{}\n".format(disprot_id, "".join(df_g['aa']), "".join(df_g['output'])))
